In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/asdff.pdf", extract_images=True)

docs = loader.load()
len(docs)

7

In [3]:
print(docs[4])

page_content='다중공선성은 독립변수간 상관관계가 있는지 나타
내는 지표이다. 본 연구에서 사용한 데이터에서는 각
독립변수는 각 column이 된다. 만약 다중공선성이 높
다면, 각 column끼리 비슷하다는 것을 의미하며, 변
수간의 차이가 없음을 의미한다. 데이터에서 column
을 너무 많이 분할하면 나타나는 현상으로, 문제가 된
다면 변수를 삭제한다거나, PCA(주성분 분석) 또는 정
규화를 이용해야한다. 상수항(const)은 독립변수 들이
함께 사용될 때 발생하는 자연스러운 결과이며, 독립
변수 간의 다중공선성 문제를 반영하지않는다. 본 연
구의 데이터에선 각 각 독립변수는 데이터에서 각각의
column을 의미한다. 실제로 중요한값은 독립변수의
VIF값들이다[7].
Table.1 VIF(Variance Inflation Factors)
위 표1은 VIF(분산팽창인수)을 나타낸다. 이 값은,
다중공선성을 나타내는 척도다. VIF값이 5보다 크면
다중 공선성에 문제가 있어 회귀 계수에 대한 신뢰할
수 없는 추정치와 해석이 발생할 수 있다. 이 값이 5
미만이면 다중공선성 문제가 크지 않다고 판단한다.
위에서 언급했듯이, 상수항이 높은 VIF값은 크게 신경
쓰지 않아도 된다[8]. 수식을 쉽게 표현하면 다음과 같
다.
식
(2)
4.3 Accuracy와 Recall
Table.2 AccuracyModel Accuracy
XGB 0.91
SVC 0.91
Ridge 0.88
LR 0.89
ADA 0.92
KNN 0.89
RF 0.89
DT 0.90
BNB 0.84
GNB 0.85
DNN 0.88
RNN+CNN 0.92
위 표2는 각각 모델별 Accuracy를 보여준다. 전체적
으로 높은 정확도를 보여준다. 수식을 쉽게 표현하면
다음과 같다.
식(3)
Table.3 Recall
위 표3은 Recall값을 즉, 재현율을 보여준다. 값들
은 소수점 두 번째 자리까지 나타낸다. 질병에 관해
서는 Recall을 고려한다. weight평가는 클래스의 데
이터 비

In [4]:
from langchain_community.document_loaders import CSVLoader

csv_files = ["table/accuracy.csv",
            "table/recall.csv",
            "table/vif.csv"]

csv_documents = []

for csv_file in csv_files:
    loader = CSVLoader(file_path=csv_file)
    csv_documents.extend(loader.load())


csv_documents


[Document(metadata={'source': 'table/accuracy.csv', 'row': 0}, page_content='\ufeff알고리즘: XGB\nAccuracy: 0.91'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 1}, page_content='\ufeff알고리즘: SVC\nAccuracy: 0.91'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 2}, page_content='\ufeff알고리즘: Ridge\nAccuracy: 0.88'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 3}, page_content='\ufeff알고리즘: LR\nAccuracy: 0.89'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 4}, page_content='\ufeff알고리즘: ADA\nAccuracy: 0.92'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 5}, page_content='\ufeff알고리즘: KNN\nAccuracy: 0.89'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 6}, page_content='\ufeff알고리즘: RF\nAccuracy: 0.89'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 7}, page_content='\ufeff알고리즘: DT\nAccuracy: 0.9'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 8}, page_content='\ufeff알고리즘: BNB\nAccuracy: 0.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50
)
documents = text_splitter.split_documents(docs)

In [6]:
len(documents)

17

In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = Chroma.from_documents(documents, embeddings)
retriever = vector_store.as_retriever()

In [8]:
retriever.invoke("본 논문에서 각각 알고리즘마다 recall값 알려줘")

[Document(metadata={'page': 4, 'source': './data/asdff.pdf'}, page_content='서는 Recall을 고려한다. weight평가는 클래스의 데\n이터 비율에 따라 가중치를 주어지는 방법이다.\nmacro평가 는 클래스의 성능 지표를 동일한 비율로\n평균을 내는 방법이다. 본 연구에서 사용한 데이터가\n5:5 데이터가 아니기 때문에, macro평균을 제시한다.\n수식은 다음과 같다.\nVIF Factor features\n33.856990 const\n1.016757 gender\n2.415670 age\n1.113385 hypertension\n1.112824 heart_disease\n1.955416 ever_married\n1.339696 work_type\n1.001191 Residence_type\n1.106672 avg_glucose_level\n1.242843 bmi\n1.216712 smoking_status\n1.087601 strokeModel Recall-weight Recall-macro\nADA 0.74 0.50\nLR 0.77 0.47\nXGB 0.74 0.49\nRF 0.84 0.47\nKNN 0.75 0.50\nSVC 0.73 0.50\nBNB 0.74 0.54\nRidge 0.75 0.50\nDT 0.70 0.55\nGNB 0.79 0.62\nDNN 0.87 0.67\nRNN+CNN 0.91 0.735'),
 Document(metadata={'page': 4, 'source': './data/asdff.pdf'}, page_content='다중공선성은 독립변수간 상관관계가 있는지 나타\n내는 지표이다. 본 연구에서 사용한 데이터에서는 각\n독립변수는 각 column이 된다. 만약 다중공선성이 높\n다면, 각 column끼리 비슷하다는 것을 의미하며, 변\n수간의 차이가 없음을 의미한다. 데이터에서 column\n을 너무 많이 분할하면 나타나는 현상으로, 문제가 된\n다면 변수를

In [9]:
csv_documents

[Document(metadata={'source': 'table/accuracy.csv', 'row': 0}, page_content='\ufeff알고리즘: XGB\nAccuracy: 0.91'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 1}, page_content='\ufeff알고리즘: SVC\nAccuracy: 0.91'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 2}, page_content='\ufeff알고리즘: Ridge\nAccuracy: 0.88'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 3}, page_content='\ufeff알고리즘: LR\nAccuracy: 0.89'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 4}, page_content='\ufeff알고리즘: ADA\nAccuracy: 0.92'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 5}, page_content='\ufeff알고리즘: KNN\nAccuracy: 0.89'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 6}, page_content='\ufeff알고리즘: RF\nAccuracy: 0.89'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 7}, page_content='\ufeff알고리즘: DT\nAccuracy: 0.9'),
 Document(metadata={'source': 'table/accuracy.csv', 'row': 8}, page_content='\ufeff알고리즘: BNB\nAccuracy: 0.

In [10]:
table_vector_store = Chroma.from_documents(csv_documents, embeddings)

In [11]:
table_retriever = table_vector_store.as_retriever()

In [12]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Please write your answer in a markdown table format with the main points.
    Be sure to include your source and page numbers in your answer.
    If I want to query the values from the Table, please use them to provide an answer.
    Answer in Korean.

    #Example Format:
    (brief summary of the answer)
    (table)
    (detailed answer to the question)

    **출처**
    - (page source and page number)

    #Question: 
    {question}

    #Context: 
    {context} 

    #Table:
    {table}

    #Answer:
    """
)

llm = ChatOpenAI(model="gpt-4o-mini")



In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = ({"context": retriever, "table": table_retriever,"question": RunnablePassthrough()}
         | prompt
         | llm
         | StrOutputParser()
         )

In [14]:
response= chain.invoke("본 논문에서 각각 알고리즘마다 recall값 알려줘")

In [15]:
print(response)

각 알고리즘의 recall 값은 다음과 같습니다.

| 모델         | Recall-weight | Recall-macro |
|--------------|---------------|---------------|
| ADA          | 0.74          | 0.50          |
| LR           | 0.77          | 0.47          |
| XGB          | 0.74          | 0.49          |
| RF           | 0.84          | 0.47          |
| KNN          | 0.75          | 0.50          |
| SVC          | 0.73          | 0.50          |
| BNB          | 0.74          | 0.54          |
| Ridge        | 0.75          | 0.50          |
| DT           | 0.70          | 0.55          |
| GNB          | 0.79          | 0.62          |
| DNN          | 0.87          | 0.67          |
| RNN+CNN      | 0.91          | 0.735         |

**출처**
- ./data/asdff.pdf, 페이지 4


In [16]:
response2 = chain.invoke("본 논문에서 제시하는 변수마다 vif값 알려줘")

In [17]:
print(response2)

다음은 본 논문에서 제시하는 각 변수의 VIF 값입니다.

| 변수        | VIF 값      |
|-------------|-------------|
| const       | 33.85699    |
| bmi         | 1.242843    |
| work_type   | 1.339696    |
| gender      | 1.016757    |

이 값들은 각 변수의 다중공선성 정도를 나타내며, VIF 값이 높을수록 다른 변수와의 상관관계가 높음을 의미합니다.

**출처**
- table/vif.csv, 페이지 번호: 1, 2, 3, 4


In [18]:
response3 = chain.invoke("본 논문에서 제시하는 모든 알고리즘의 Accuracy값 알려줘")

In [19]:
print(response3)

알고리즘의 Accuracy 값은 다음과 같습니다.

| 알고리즘  | Accuracy |
|-----------|----------|
| ADA       | 0.92     |
| LR        | 0.89     |
| DNN       | 0.88     |
| SVC       | 0.91     |

각 알고리즘의 정확도는 ADA가 0.92로 가장 높고, DNN이 0.88로 가장 낮습니다. LR과 SVC는 각각 0.89와 0.91의 정확도를 보입니다.

**출처**
- table/accuracy.csv, 페이지 번호: 1, 3, 4, 10


In [16]:
response4 = chain.invoke("")
print(response4)

모델링 과정에서 사용된 알고리즘 및 방법론에 대한 설명입니다.

| 주요 사항                   | 내용                                                      |
|----------------------------|---------------------------------------------------------|
| 데이터 전처리               | 표준화(standard scaler) 사용, train/valid/test 데이터 비율 0.6/0.2/0.2 |
| 머신러닝 알고리즘          | Logistic Regression, Light GBM, CatBoost, Naive Bayes, RandomForest, Decision Tree |
| 딥러닝 모델                | DNN, RNN과 CNN 결합, 5개 은닉층                      |
| 활성화 함수                | 숨겨진 층: ReLU, 출력 층: sigmoid                     |
| 최적화 알고리즘            | Adam 사용, 기울기 업데이트                            |
| 손실 함수                  | Binary Cross Entropy                                    |
| RNN 및 CNN 구조            | RNN: 임베딩 차원 128, LSTM; CNN: Conv1d, 커널 수 32, 커널 크기 5 |

**출처**
- ./data/asdff.pdf (페이지 1)
